In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import matplotlib.pyplot as plt
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv',sep='\t',error_bad_lines=False )

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [4]:
df_star_review = df[['star_rating','review_body']]


In [5]:
df_star_review.head()

,star_rating,review_body
0,5,As described.
1,5,It works as advertising.
2,5,Works pissa
3,1,Did not work at all.
4,5,Works well. Bass is somewhat lacking but is pr...


In [6]:
df_star_review_sampled = df_star_review.sample(100_000)

In [7]:
df_star_review_sampled.dropna(inplace=True)

blanks = []  # start with an empty list

for i,lb,rv in df_star_review_sampled.itertuples():  
    if type(rv)==str:            
        if rv.isspace():        
            blanks.append(i)     

df_star_review_sampled.drop(blanks, inplace=True)

In [8]:
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [9]:
 review_body = df_star_review_sampled['review_body']
 review_body = str(review_body).encode('utf-8')
df_star_review_sampled['scores'] = df_star_review_sampled[str('review_body')].apply(lambda review_body: sid.polarity_scores(review_body))

df_star_review_sampled.head(100)


,star_rating,review_body,scores
98188,1,Snapped at one end in less than a day of use. Can't return for I am using it outside of USA.,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
41771,1,Defective!! Right ear doesn't work and yes the cords are plugged in all the way. I tried multiple sources. Pretty sad considering how much they charged for them. Do NOT buy.,"{'neg': 0.274, 'neu': 0.644, 'pos': 0.082, 'compound': -0.7537}"
31690,1,"I purchased 5 of these to power hobby projects. I was using three constantly for about a month and two of them failed and one would have failed within a very short time had I not unplugged it. When I opened the case on the one that had not failed I could see the electrolytic capacitor bulging. The problem is a small surface mount resistor on the back of the circuit board, marked 680, which ...","{'neg': 0.071, 'neu': 0.879, 'pos': 0.049, 'compound': -0.597}"
113259,5,thank you,"{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}"
336977,4,It's very comfortable.,"{'neg': 0.0, 'neu': 0.358, 'pos': 0.642, 'compound': 0.5563}"
...,...,...,...
91192,2,Worked great for about 3 weeks...then they would no longer charge...had to send the defective headset to Monoprice for a new replacement...I am still awaiting the replacement...it has now been two weeks gone and I do not have the item I purchased...buyer beware!!!,"{'neg': 0.13, 'neu': 0.781, 'pos': 0.089, 'compound': 0.2206}"
353523,3,This product produces a very pleasing HD image. Unfortunately the reception is very sensitive to movement. It's advertised as a multidirectional device that does not require precise directional positioning. I find the slightest movement can cause pixelization or complete loss of a channel. I've had to move the devise several times in order to find the location that brings in all local stat...,"{'neg': 0.085, 'neu': 0.83, 'pos': 0.085, 'compound': 0.2467}"
245600,5,very good,"{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'compound': 0.4927}"
83202,5,"super great, works very well. sound is awesome.","{'neg': 0.0, 'neu': 0.216, 'pos': 0.784, 'compound': 0.9381}"


In [11]:
from google.colab import files
import pandas as pd
df_star_review_sampled.head(10000).to_csv('PostAnalysis.csv')
files.download('PostAnalysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
sample_df=pd.read_csv('PostAnalysis.csv')

In [14]:
one_star_df=sample_df.loc[sample_df['star_rating'] == 1]
two_star_df=sample_df.loc[sample_df['star_rating'] == 2]
three_star_df=sample_df.loc[sample_df['star_rating'] == 3]
four_star_df=sample_df.loc[sample_df['star_rating'] == 4]
five_star_df=sample_df.loc[sample_df['star_rating'] == 5]

In [15]:
def plot_neg_scores(df):
  out = pd.cut(df['neg'], bins=[0, 0.1, 0.2,0.3, 0.4,0.5, 0.6,0.7, 0.8, 0.9, 1], include_lowest=True)
  ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(16,6))
  plt.show()

In [16]:
def plot_pos_scores(df):
  out = pd.cut(df['pos'], bins=[0, 0.1, 0.2,0.3, 0.4,0.5, 0.6,0.7, 0.8, 0.9, 1], include_lowest=True)
  ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(16,4))
  plt.show()

In [17]:
def plot_neu_scores(df):
  out = pd.cut(df['neu'], bins=[0, 0.1, 0.2,0.3, 0.4,0.5, 0.6,0.7, 0.8, 0.9, 1], include_lowest=True)
  ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(16,4))
  plt.show()

In [18]:
def plot_com_scores(df):
  out = pd.cut(df['compound'], bins=[-1, -0.5, 0, 0.5, 1], include_lowest=True)
  ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(16,4))
  plt.show()

In [ ]:
plot_neg_scores(one_star_df)

In [ ]:
plot_neg_scores(two_star_df)

In [ ]:
plot_neg_scores(three_star_df)

In [ ]:
plot_pos_scores(one_star_df)

In [ ]:
plot_neu_scores(one_star_df)

In [ ]:
plot_pos_scores(two_star_df)

In [ ]:
plot_neu_scores(two_star_df)

In [ ]:
plot_pos_scores(three_star_df)

In [ ]:
plot_neu_scores(three_star_df)

In [ ]:
plot_neg_scores(four_star_df)

In [ ]:
plot_pos_scores(four_star_df)

In [ ]:
plot_neu_scores(four_star_df)

In [ ]:
plot_neg_scores(five_star_df)

In [ ]:
plot_pos_scores(five_star_df)

In [ ]:
plot_neu_scores(five_star_df)

In [ ]:
counts=[0,0,0,0,0]
for i,x in sample_df.iterrows():
  counts[classify_rec(x)-1]=counts[classify_rec(x)-1]+1

In [ ]:
counts

In [ ]:
df  = pd.read_csv("PostAnalysis.csv").sample(400)
df.plot(kind='scatter',x='star_rating',y='compound') # scatter plot